In [67]:
q = 2 
m = 6 
p = x^6 + x^4 + x^3 + x + 1
F.<a> = GF(q^m, modulus=p)
R.<x> = PolynomialRing(F) 

Suppose 
    m = 6 </br>
    a is prim. elem  </br>
    q = 2  </br>
    p(a) = a^6 + a^4 + a^3 + a + 1 


In [68]:
g = x^7 + x + 1
g_t = [1, 0, 0, 0, 0, 0, 1] # hardcoded for now: 
g_t

[1, 0, 0, 0, 0, 0, 1]

In [69]:
n = q^m 
T = g.degree() 
T

7

In [70]:
L =[a^i for i in range(1, n + 1) if g(a^i) != 0 ]
len(L)

64

In [71]:
# lookup_alpha
# L =[a^i for i in range(1, n+1) if g(a^i) != 0 ]
print(len(L))
Z = matrix(F, [[(1 / g(el)) if i == j else 0 for i, el in enumerate(L)] for j in range(n)])
Y = matrix(F, [[a_i^t for a_i in L] for t in range(T)])
X = [[0] * len(g_t) for _ in range(len(g_t))]
for i in range(len(g_t)): 
    for j in range(len(g_t)): 
        if j >= i: 
            X[j][i] = g_t[j-i]
X = matrix(F, X) 
print(f"dimension Z {Z.dimensions()}")
print(f"dimension Y {Y.dimensions()}")
print(f"dimension X {X.dimensions()}")

64
dimension Z (64, 64)
dimension Y (7, 64)
dimension X (7, 7)


In [169]:
H = X*Y*Z.T
H.dimensions()
H_ = [] 
for row in H.T: 
    vec = vector(F, list([vector(i) for i in row]))
    sss = [] 
    for r in vec: 
        sss += r 
    H_.append(sss) 
H_ = matrix(H_).T
H_.dimensions()

(42, 64)

In [170]:
generator_matrix = H_.right_kernel_matrix()
null_basis = generator_matrix*H.T 
if any(null_basis): 
    print("failed") 
else:
    print("worked!") 
generator_matrix.dimensions()

worked!


(22, 64)

Sanity check:

In [171]:
C = codes.GoppaCode(g, L) 
print(f"{C.minimum_distance()}") 
print(f"{C.distance_bound()}")
print(f"{C.generator_matrix().dimensions()}")
HHH = C.parity_check_matrix()
HHH.row_space() == H_.row_space()


2
8
(22, 64)


True

In [172]:
text = "TIMEFLIESQUICKLY" 

def text2bin(msg):
    return "".join(f"{ord(i):08b}" for i in msg) 
def bin2text(bin_s): 
    return "".join([chr(int(bin_s[i:i+8], 2)) for i in range(0, len(bin_s), 8)])
def to_field_element(string):
    """Binary String to field Element"""
    elem = 0
    n = len(string)
    for i in range(n):
        coeff = int(string[n-i-1])
        elem += coeff * a^i # coeff is 0 or 1
    return elem

def encodeGoppa(m):
    bin_m = text2bin(m) 
    m = m + "0"*(128 - len(bin_m)) 
    bin_m_chunks = [bin_m[i:i+64] for i in range(0, len(bin_m), 64)]
    half_blocks = []
    for half in bin_m_chunks:
        r_i = 0 
        for i, value in enumerate(half): 
            r_i += int(value) / (x - (a^i)) 
        if r_i.mod(g) == 0: # 
            half_blocks.append(r_i)
        else: 
            raise Exception("Was not zero!")
        r_i = 0 
    return half_blocks


code_word = encodeGoppa(text)

In [173]:
generator_matrix

22 x 64 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)

# Goppa decoding 

Pattersons algortihm for decoding. Corrects t design errors.  
Patterson converts a syndrome to a vector of erros. The syndrome of a binary word c = (c_0, ..., c_n-1) is 

In [197]:
import random 
# r = vector(F, (0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1))
r = vector(GF(2), [random.choice([0, 1]) for _ in range(64)])
# r =  vector(F, generator_matrix.T * r)
# rr = vector(GF(2), valid_r * generator_matrix.T)
#c = generator_matrix * r
# r = vector(F, (1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1))
len(r)
n = 7 

In [198]:
# r = vector(F, r * generator_matrix.T)

In [199]:
h = prod((x-value) for value in L)
h

x^64 + a*x^63 + x + a

In [200]:
# input to goppa decoding is a binary vector, of vector in base field 
def find_syndromes(received_vector):  
    codelocators = [] 
    F_2m = g.base_ring()
    PR_f_2m = g.parent() 
    X = PR_f_2m.gen() 
    codelocators = [a^(i+1) for i in range(n-1)]
    codelocators.append(F_2m(0))
    syndrome_vector = vector(PR_f_2m, [(x - codelocators[i]).inverse_mod(g) for i in range(n)])
    return syndrome_vector * received_vector
    
def find_syndromesV2(received): 
    """
    Syndrome of r is the result of H x r where H is the 
    parity check matrix. r belongs to the code if its 
    syndrome equals to the zero vector.
    """
    
    return H_ * received 
    
def GoppaDecoding(s_x, g_x):
    
    pass 


In [201]:
def from_list_to_poly(generic_list): 
    return sum([x for x in generic_list])

In [206]:
# assume f(x) = x^2 +1 = [1, 0, 1]
f_x = x^5 + x^4+ x^3 + x^2 + x + 1
def Berlekamp_decoding(f_x):  
    """
    input: f(x): sum_{i=0}^{n} a_i*x^i = a_0 + a_1*x + ... + a_n-1 x^n-1
    return: sigma^
    """
    # f_x = vector(F, f_x)
    f = (f_x).dict()
    # assumen the first nonzero coefficient of f is one  
    sigma, tau, omega, gamma, = dict(), dict(), dict(), dict()
    
    D, B = dict(), dict()
    # quick fix 
    D[-1], B[-1] = 0, 0
    
    if f[0] == 1: 
        sigma[0], tau[0], omega[0], gamma[0] = 1, 1, 1, 0
        D[0], B[0] = 0, 0, 
     
    else: 
        sigma[0], tau[0], omega[0], gamma[0] = 1, 1, 0, -1
        D[0], B[0] = 0, 1
    
    k = 0 
    while k <= n - 2: 
        # take delta k as the coefficient of x^k in f*sigma_k
        fdelta_k = (f_x * sigma[k])
        
        delta_k = fdelta_k.list()[k]
        # now find coefficient of x^k from tmp 
        sigma[k+1] = sigma[k] - delta_k * x * tau[k]
        omega[k+1] = omega[k] - delta_k * x * gamma[k]
        
        if delta_k == 0 or (D[k] > ((k + 1) / 2)) or (D[k] == ((k + 1) / 2) and B[k] == 0): 
            D[k + 1] = D[k]
            B[k + 1] = B[k]
            tau[k+1] = x * tau[k]
            gamma[k+1] = x * gamma[k]
        
        elif delta_k != 0 and (((B[k-1] == 0) and D[k-1] <= (k - floor(n/2) - 1)) or (B[k-1])==1 and D[k-1] <= k - floor((n-1)/2) - 1):
            sigma_hat = x ^(n-k) * sigma[k]
            omega_hat = x ^(n-k) * omega[k]
            N_hat = n - k + D[k - 1] + 1 - B[k - 1]
            # assert (f_x*sigma_hat) == (omega_hat % x^n)
            return sigma_hat, omega_hat, N_hat
        else:  
            D[k+1] = (k+1) - D[k]
            B[k+1] = 1 - B[k]
            tau[k+1] = sigma[k] / delta_k
            gamma[k+1] = k / delta_k 
        k = k + 1 
    sigma_hat = sigma[n-1]
    omega_hat = omega[n-1]
    N_hat     = D[n-1] + 1 - B[n-1]
    # assert (f_x*sigma_hat) == (omega_hat % x^n)
    return sigma_hat, omega_hat, N_hat
 
# f = x^(n-1) + x +1 
# var = Berlekamp_decoding(f_x)



In [230]:
def PattersonsDecoding(s_x, g_x): 
    a_i = []
    for i in range(n-1): 
        tmp = ((x^i) * s_x) % g_x 
        tmp = tmp.list() 
        a_i.append(tmp[-1])
    f_x = from_list_to_poly([a_i[i] * x^i for i in range(n-1)])
    sigma_y, omega, N_hat = Berlekamp_decoding(f_x)
    sigma_x = sum([sigma_y[i]*x^(N_hat - i) for i in range(sigma_y.degree())])
    omega = (s_x * omega) % g_x 
    print(3 <= 3)
    print(int(sigma_x.degree()) <= int(floor(n/2)))
    # assert sigma_x.degree() <= floor(n/2)
    print((s_x * sigma_x)%g_x == N_hat % g_x)
    return sigma_x, omega 
    
s_x = find_syndromes(vector(F, [random.choice([0, 1]) for _ in range(n)]))
PattersonsDecoding(s_x, g)


True
False
False


(x^4 + (a^5 + a^4 + a^3 + 1)*x^3 + (a^4 + a^2)*x^2,
 a^4*x^6 + (a + 1)*x^5 + (a^5 + a^3 + a^2 + a + 1)*x^4 + (a^5 + a^4 + a^2 + a + 1)*x^3 + (a^5 + a^3 + a^2)*x^2 + a^5*x + a^4)

In [211]:
def _split(p):
    # split polynomial p over F into even part po
    # and odd part p1 such that p(z) = p2 (z) + z p2 (z)
    Phi = p.parent()
    p0 = Phi([sqrt(c) for c in p.list()[0::2]]);
    p1 = Phi([sqrt(c) for c in p.list()[1::2]]);
    return (p0,p1);

def _g_inverse(p):
    (d, u, v) = xgcd(p, g)
    return u.mod(g)


def PattersonsDecodingForBinaryGoppaCodes(s_x, g_x): 
    if not any(s_x.list()): return "syndrome zero" 
    h = s_x.inverse_mod(g_x) 
    if h == x: 
        sigma_x = x 
    else: 
        #  
        (g0, g1) = _split(g_x)
        sqrt_X = g0 * _g_inverse(g1) 
        (T0, T1) = _split(_g_inverse(s_x) - x) 
        d = (T0 + sqrt_X * T1).mod(g_x)
        sigma, N1 = PattersonsDecoding(s_x, g_x) 
        
        print((d*d)%g_x == (h+x) % g_x)
        print(sigma)
        print(sigma.degree() <= n/2)
        print((d*sigma)%g_x == n % g_x)
        
PattersonsDecodingForBinaryGoppaCodes(s_x, g) 

    

True
x^4 + (a^4 + 1)*x^3 + (a^4 + a + 1)*x^2
False
False
